### Hotel in Jeju Data
- '부킹닷컴'의 제주 내 호텔 성급, 가격, 위치, 후기를 스크래핑합니다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import InvalidSessionIdException
from time import sleep
import re
import json
import pandas as pd

In [2]:
url = 'https://www.booking.com/searchresults.ko.html?label=gen173nr-1DCAEoggI46AdIM1gEaH2IAQGYARe4AQfIAQzYAQPoAQGIAgGoAgO4Ar3AsYMGwAIB0gIkNjQ0OTNkZTgtOWUzNC00ODBjLTlmZjktYjA1ZjcwOTFmN2Vk2AIE4AIB&sid=febadbcd7e51aee26acbfeeea6e8299e&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.ko.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaH2IAQGYARe4AQfIAQzYAQPoAQGIAgGoAgO4Ar3AsYMGwAIB0gIkNjQ0OTNkZTgtOWUzNC00ODBjLTlmZjktYjA1ZjcwOTFmN2Vk2AIE4AIB%3Bsid%3Dfebadbcd7e51aee26acbfeeea6e8299e%3Btmpl%3Dsearchresults%3Bclass_interval%3D1%3Bdest_id%3D4170%3Bdest_type%3Dregion%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dregion%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dindex%3Bsrc_elem%3Dsb%3Bsrpvid%3Dadca5de230db0091%3Bss%3D%25EC%25A0%259C%25EC%25A3%25BC%25EB%258F%2584%252C%2520%25EB%258C%2580%25ED%2595%259C%25EB%25AF%25BC%25EA%25B5%25AD%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84&is_ski_area=0&ssne=%EC%A0%9C%EC%A3%BC%EB%8F%84&ssne_untouched=%EC%A0%9C%EC%A3%BC%EB%8F%84&dest_id=4170&dest_type=region&checkin_year=2021&checkin_month=4&checkin_monthday=24&checkout_year=2021&checkout_month=4&checkout_monthday=25&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
path = '/Users/seyoung/2021/dss17/eda_etc/chromedriver'
# path_f = '/Users/seyoung/2021/dss17/eda_etc/geckodriver'
driver = webdriver.Chrome(path)
# driver = webdriver.Firefox(executable_path=path_f)
driver.get(url)

In [3]:
page_bar = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul')[0]
page_bar

<selenium.webdriver.remote.webelement.WebElement (session="d6c56f1c0ee965e3dfcb34bcc0b45d3b", element="71356cfc-d495-4117-ae49-78d8fc273809")>

In [4]:
# 전체 페이지 개수 확인
try:
    pages = page_bar.find_elements_by_css_selector('a > div.bui-u-inline')
    last_pages = ''

    for idx, page in enumerate(pages):
        print(page.text)
        if idx == len(pages)-1: # 마지막 페이지
            last_pages = page.text
except:
    pages = 0
    last_pages = 0    

1
2
3
4
5
19


In [5]:
# 현재 페이지 번호
def get_curr():
    try:
        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0].text
    except:
        curr_page = 0
    return int(curr_page)

In [6]:
# 성급 (성급이 표시되지 않은 호텔 제외)
def get_stars(listed):
    stars = []

    for i in range(1, len(listed)):
        try:
            like = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/span/span[1]/span/span')
            a = like.get_attribute('aria-label')
            stars.append(a[0])
            
        # 성급이 표시되지 않는 호텔 
        except Exception as NoSuchElementException:
            stars.append('')
    
    return stars

In [7]:
# 가격
def get_prices(listed):
    prices = []
    
    for i in range(1, len(listed)):
        try:
            price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/div')[0]
            base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/span')[0]
            
            # strong label로 표시된 가격 포함
            if base_price.text == '':
                base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[2]/div/table/tbody/tr/td[2]/div[2]/strong/label')[0]
            
            prices.append(base_price.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            prices.append('')
            
    return prices

In [8]:
# 평점
def get_grades(listed):
    grades = []
    
    for i in range(1, len(listed)):
        try:
            grade = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[1]/div[2]/div/div/a/div/div[1]')[0]
            grades.append(grade.text)
            
        except Exception as IndexError:
            grades.append('')
            
    return grades

In [9]:
# 호텔명
def get_names(listed):
    names = []
    
    for i in range(1, len(listed)):
        try:
            name = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]')[0]
            names.append(name.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            names.append('')
            
    return names

In [23]:
# 이용객 후기
def get_reviews():
    good_reviews, bad_reviews, res = [], [], []
    good_review, bad_review = '', ''
    review = driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li')
    
    for i in range(len(review)):
        
        try:
            good_review += driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li['+ str(i) +']/div/div[2]/div[2]/div[2]/div/div[1]/p/span[3]')[0].text +'\n'
        
        except Exception as IndexError:
            good_review += ''
            
            try:
                bad_review += driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li['+ str(i) +']/div/div[2]/div[2]/div[2]/div/div[2]/p/span[3]')[0].text +'\n'
            
            except Exception as IndexError:
                bad_review += ''
        
    good_reviews.append(good_review)
    bad_reviews.append(bad_review)
    
    res.append(good_reviews)
    res.append(bad_reviews)
    
    return res

In [12]:
# 주소
def get_addrs():
    
    addr = driver.find_elements_by_xpath('//*[@id="showMap2"]/span[1]')[0]

    return addr.text

In [22]:
# 새 탭 열기
def open_newTab(listed):
    
    addrs, res, res_reviews = [], [], []
    
    for i in range(1, len(listed)):

        try:
            href = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a')
            
        except Exception as NoSuchElementException:
                addrs.append('')
                res_reviews.append('')
                continue
        else:
            href.send_keys(Keys.CONTROL + '\n') 

           # 새로운 탭으로 이동 (2번째 탭)
            driver.switch_to.window(driver.window_handles[1])  

            res_addrs = get_addrs()
            addrs.append(res_addrs)

            # 이용 후기 메뉴로 이동
            comment_btn = driver.find_element_by_xpath('//*[@id="show_reviews_tab"]')
            driver.execute_script("arguments[0].click();", comment_btn)

            res_reviews = get_reviews()
            res.append(res_reviews)

            driver.implicitly_wait(1)
            time.sleep(1+random.uniform(0,1))

            driver.close()
            driver.switch_to_window(driver.window_handles[0])
        
    return list(zip(res, addrs))

In [14]:
def data_scraping():
    res_stars, res_names, res_prices, res_grades, res_addrs, res_reviews = [], [], [], [], [], []
    
    root_div = driver.find_elements_by_css_selector('#hotellist_inner')[0]
    # 한 페이지당 호출되는 목록 수
    listed = root_div.find_elements_by_xpath('//*[@id="hotellist_inner"]/div')
    
    res_stars = get_stars(listed)
    res_names = get_names(listed)
    res_prices = get_prices(listed)
    res_grades = get_grades(listed)
    
    res = open_newTab(listed)
    
    for i in range(len(res)):
        res_addrs.append(res[i][1])
        res_reviews.append(res[i][0][0])
    
    return list(zip(res_names, res_stars, res_prices, res_grades, res_addrs, res_reviews))

# data_scraping()

In [15]:
def get_hotelDatas():
    raw_datas = []
    
    # 스크래핑 페이지 범위 (현재 페이지 ~ 마지막 페이지)
    while get_curr() <= int(last_pages):
        # scraping
        result = data_scraping()
        raw_datas.append(result)

        if get_curr() == int(last_pages):
            break

        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
        print(curr_page.text,"페이지 완료")

        move_next = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__item.bui-pagination__next-arrow > a')[0]
        move_next.send_keys('\n')

        driver.implicitly_wait(1)
        time.sleep(1+random.uniform(0,1))
        
    return raw_datas

In [16]:
# 기존 요금, 현재 요금 데이터 파싱
def price_parsing(datas):
    target_prices = []
    
    for i in range(len(datas[0])):
        target_prices.append(datas[0][i][2])

    prices = [] 

    for i in range(len(target_prices)):
        re_prices = re.findall('\d+', target_prices[i])
        base_curr_prices = [] 

        if len(re_prices) == 0:
            base_curr_prices.append('')
            base_curr_prices.append('')
            
        elif len(re_prices) == 2:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append('')
            
        else:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append(re_prices[2] + re_prices[3])

        prices.append(base_curr_prices)

    return prices

In [17]:
datas = get_hotelDatas()
re_prices = price_parsing(datas)

raw_dicts = {'hotel_name':'', 'star':'', 'base_price':'', 'cur_price':'', 'grade':'', 'review':'', 'address':''}
raw_datas = []
hotelName_idx, star_idx, grade_idx, addr_idx, review_idx, = 0, 1, 3, 4, 5

for i in range(len(datas[0])):
    
    raw_dicts['hotel_name'] = datas[0][i][hotelName_idx]
    raw_dicts['star'] = datas[0][i][star_idx]
    raw_dicts['base_price'] = re_prices[i][0]
    raw_dicts['cur_price'] = re_prices[i][1]
    raw_dicts['grade'] = datas[0][i][grade_idx]
    raw_dicts['review'] = datas[0][i][review_idx][0]
    raw_dicts['address'] = datas[0][i][addr_idx]
    
    raw_datas.append(raw_dicts.copy())
    
raw_datas

1 페이지 완료
2 페이지 완료
3 페이지 완료
4 페이지 완료
5 페이지 완료
6 페이지 완료
7 페이지 완료
8 페이지 완료
9 페이지 완료
10 페이지 완료
11 페이지 완료
12 페이지 완료
13 페이지 완료
14 페이지 완료
15 페이지 완료
16 페이지 완료
17 페이지 완료
18 페이지 완료


[{'hotel_name': '신신호텔 천지연',
  'star': '4',
  'base_price': '',
  'cur_price': '',
  'grade': '8.3',
  'review': '',
  'address': '서귀포, 서문로 42-1'},
 {'hotel_name': '롯데호텔 제주',
  'star': '5',
  'base_price': '617100',
  'cur_price': '',
  'grade': '8.6',
  'review': '객실이 넓고 깨긋해서 아주 좋았어요.가구도앤틱분위기로 아늑하면서\n서귀포 이동이 가까웠고 조식이 너무 맛있었음\n산책로도 좋았고 그외에 서비스가 만족스러웠음\n직원들이 매우 친절했습니다\n위치는 별다섯개! 서비스도 별다섯개! 조식도 별다섯개!\n조식이너무 훌륭하고, 수영장도 좋았어요\n가든뷰의 룸이었는데 예쁜 조경과 수영장이 보이고 저녁 공연까지 들려서 잠시나마 외국에 온 느낌이었습니다. 체크인 때 꽃 한송이를 함께 주시는 센스하며 직원들 한분 한분 어찌나 친절하신지 이래서 특급호텔이구나 생각되었습니다. 방이 넓고 쾌적했으며 온도 조절도 조용히 잘 되었습니다. 조식은 7시30분~8시쯤 갔는데 많이 붐비지 않았고 음식 종류도 많아 맛있게 먹을 수 있었습니다. 수영장 온도도 좋았고 선베드도 충분해서 즐겁게 물놀이를 할 수 있었습니다. 롯데호텔 튜브 사왔는데 앞으로고 요긴하게 잘 사용할 것 같습니다.\n산책로를 따라 해변까지 다녀왔는데 새소리도 많이 들리고 바다와 어우러진 풍광이 멋졌습니다.\n유럽만 6번 다녀올 정도로 여행을 좋아하는 사람인데 코로나로 신혼여행을 제주로 가게되어 무척 속상했습니다. 근데 롯데호텔에 투숙하며 많이 힐링되었습니다. 정말 감사드립니다!!!\n시설, 직원들의 서비스, 숙소 상태, 수영장 다 좋음\n직원분들이 정말 친절하고 시설도 깨끗하고 다 좋았습니다. 그리고 오션뷰 방을 잡았는데 미리 전화를 통해 고층 배정을 요청했어요. 경치도 좋고 테라스가 있어서 편하고 좋았습니다.\n

In [20]:
def get_fields(raw_datas):

    name = [raw_datas[i]['hotel_name'] for i in range(len(raw_datas))]
    star = [raw_datas[i]['star'] for i in range(len(raw_datas))]
    basePrice = [raw_datas[i]['base_price'] for i in range(len(raw_datas))]
    currentPrice = [raw_datas[i]['cur_price'] for i in range(len(raw_datas))]
    grade = [raw_datas[i]['grade'] for i in range(len(raw_datas))]
    review = [raw_datas[i]['review'] for i in range(len(raw_datas))]
    address = [raw_datas[i]['address'] for i in range(len(raw_datas))]
    
    data_pd = pd.DataFrame({'name': name, 'star': star, 'basePrice': basePrice, 'currentPrice': currentPrice, 'grade': grade, 'review': review, 'address': address })
    
    return data_pd

res_jeju_hotel = get_fields(raw_datas)
res_jeju_hotel

,name,star,basePrice,currentPrice,grade,review,address
0,신신호텔 천지연,4,,,8.3,,"서귀포, 서문로 42-1"
1,롯데호텔 제주,5,617100,,8.6,객실이 넓고 깨긋해서 아주 좋았어요.가구도앤틱분위기로 아늑하면서\n서귀포 이동이 가...,"서귀포, 중문관광로 72번길 35"
2,랜딩관 제주신화월드 호텔앤리조트,5,186744,172040,8.4,아기 둘과 함께 여행이라서 짐이 많았는데 직원분이 룸까지 함께 옮겨 주셔서 너무너무...,"서귀포, 신화역사로 304번길 , 38"
3,메종 글래드 제주,5,235142,152750,8.1,업그레이드되서 넘나좋았\n친절하게 잘 해주었음\n위치가 시내에 가까움\n시설이 좀 ...,"제주, 노연로 80"
4,메리어트관 제주신화월드 호텔앤리조트,5,319397,303050,8.4,직원들이 매우 친절합니다. 청소하시는 분들은 먼저 인사도 건네주시고 너무 좋았습니다...,"서귀포, 신화역사로 304번길 , 38"
5,롯데시티호텔 제주,4,177650,,8.6,조식이 정말 너무 괜찮았어요! 홍콩에서 밥먹는듯한 기분? 이 드는 빌딩숲속의 아침이...,"제주, 도령로 83"
6,호텔 리젠트 마린 더블루,4,217500,87000,8.2,"오션뷰가 정말 좋았구,숙소도 너무깨끗하고 청결해서 놀랬어요~걸어서 칠성흑돼지거리랑 ...","제주, 서부두2길 20"
7,호텔 휘슬락,4,300018,75000,8.0,각 방마다 발코니가 있고 흑돼지거리와 동문시장이 가볍게 걸어갈 수 있는 거리에 있어...,"제주, 서부두2길 26"
8,그랜드 하얏트 제주,5,,,8.7,직원분들 친절하고 특히 청소해주시는 한분 감동했어요 \n38층 포차 특히 좋아요 바...,"제주, 노연로 12"
9,신화관 제주신화월드 호텔 앤 리조트,5,220274,181830,8.3,"매우 편안하고 깨끗했습니다\n볼링장, 오락실, 워터파크 등 밖에 나가지 않아도 놀거...","서귀포, 신화역사로 304번길 38"


In [21]:
res_jeju_hotel.to_csv("jeju_hotel_datas.csv", encoding='utf-8-sig', header=True, mode='a') # 이어서 저장

---